In [1]:
import hopsworks

proj = hopsworks.login()
fs = proj.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://hopsworks0.logicalclocks.com/p/122
Connected. Call `.close()` to terminate connection gracefully.


In [2]:
!pip install sentence_transformers

In [3]:
import logging

In [4]:
logger = logging.getLogger()
logger.setLevel(logging.WARN)

In [5]:
logging.getLevelName(logging.getLogger().getEffectiveLevel())

'WARNING'

In [6]:
import pandas as pd

df = pd.read_csv("Articles.csv", encoding='utf-8', encoding_errors='ignore')
df = df[:20]

In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [8]:
df["Heading"][0]

'sindh govt decides to cut public transport fares by 7pc kti rej'

In [9]:
len(model.encode(df["Heading"][0]))

384

In [10]:
embeddings_heading = model.encode(df["Heading"])
embeddings_body = model.encode(df["Article"])
df["embedding_headding"] = pd.Series(embeddings_heading.tolist())
df["embedding_body"] = pd.Series(embeddings_body.tolist())
df["id1"] = list(range(len(df)))

In [11]:
df.head()

,Article,Date,Heading,NewsType,embedding_headding,embedding_body,id1
0,KARACHI: The Sindh government has decided to b...,1/1/2015,sindh govt decides to cut public transport far...,business,"[-0.002680476987734437, 0.010348751209676266, ...","[0.015569651499390602, 0.05348442867398262, 0....",0
1,HONG KONG: Asian markets started 2015 on an up...,1/2/2015,asia stocks up in new year trad,business,"[-0.028134549036622047, -0.045254770666360855,...","[0.011652727611362934, 0.0017589479684829712, ...",1
2,HONG KONG: Hong Kong shares opened 0.66 perce...,1/5/2015,hong kong stocks open 0.66 percent lower,business,"[0.09226575493812561, -0.0026451086159795523, ...","[0.06687025725841522, -0.042929936200380325, 0...",2
3,HONG KONG: Asian markets tumbled Tuesday follo...,1/6/2015,asian stocks sink euro near nine year,business,"[-0.0146249420940876, 0.0414726585149765, 0.06...","[-0.017030566930770874, -0.00658829789608717, ...",3
4,NEW YORK: US oil prices Monday slipped below $...,1/6/2015,us oil prices slip below 50 a barr,business,"[-0.04315836355090141, 0.002260032342746854, 0...","[-0.00556448008865118, 0.013064294122159481, 0...",4


In [12]:
import random
df_view = pd.DataFrame({"id1": list(range(len(df))), "view_cnt": [random.randint(0, 100) for i in range(len(df))]})

In [13]:
df_view

,id1,view_cnt
0,0,81
1,1,14
2,2,3
3,3,94
4,4,35
5,5,31
6,6,28
7,7,17
8,8,94
9,9,13


In [14]:
version = 1

In [15]:
from datetime import datetime

In [16]:
from hsfs import embedding
# TODO: check if index exist
emb = embedding.EmbeddingIndex()
# TODO: add model uri
# TODO: validate embedding col type
emb.add_embedding("embedding_body", len(df["embedding_body"][0]))
emb.add_embedding("embedding_headding", len(df["embedding_headding"][0]))
# TODO: embedding --> embedding_index

news_fg = fs.get_or_create_feature_group(
    name="news_fg_3",
#     event_time="ts",
    embedding_index=emb,
    primary_key=["id1"],
    version=version,
    online_enabled=True,
)
df_filter = df[[col for col in df.columns]]
# df_filter["ts"] = [datetime.now() for i in range(len(df_filter))]
news_fg.insert(df_filter, write_options={"start_offline_backfill": True})

Feature Group created successfully, explore it at 
https://hopsworks0.logicalclocks.com/p/122/fs/70/fg/41


Uploading Dataframe: 0.00% |          | Rows 0/20 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: news_fg_3_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://hopsworks0.logicalclocks.com/p/122/jobs/named/news_fg_3_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f43578ff4f0>, None)

In [17]:
view_fg = fs.get_or_create_feature_group(
    name="view_fg",
    primary_key=["id1"],
    version=version,
    online_enabled=True,
)

view_fg.insert(df_view, write_options={"start_offline_backfill": True})

Feature Group created successfully, explore it at 
https://hopsworks0.logicalclocks.com/p/122/fs/70/fg/42


Uploading Dataframe: 0.00% |          | Rows 0/20 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: view_fg_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://hopsworks0.logicalclocks.com/p/122/jobs/named/view_fg_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f43578fd180>, None)

## Search News

In [19]:
news_fg = fs.get_feature_group("news_fg_3", version=version)

In [20]:
news_description = "news about europe"

In [21]:
news_fg.find_neighbors(model.encode(news_description), k=1, col="embedding_headding")

[(-0.28375966584236956,
  ['HONG KONG: Asian markets tumbled Tuesday following painful losses in New York and Europe while the euro sat near nine-year lows as political uncertainty in Greece fanned renewed fears it could leave the eurozone.Oil prices, which fell below the psychological $50 a barrel mark in US trade, edged up marginally but remained under pressure owing to a global supply glut, weak demand and a stronger dollar.Tokyo tumbled 2.50 percent, Hong Kong lost 0.89 percent, Sydney eased 1.67 percent, Seoul was 1.30 percent lower while Shanghai reversed earlier losses to gain 0.70 percent.The first full week of the new year got off to a traumatic start for dealers as they bet a January 25 general election in Greece will see a victory for the the left-wing Syriza party.Markets fear the party will roll back austerity measures required under the IMF-EU bailout of the country, which could in turn lead it to exit the eurozone.The year is "barely three trading days old and already th

In [22]:
fv = fs.get_or_create_feature_view("news", version=version, query=news_fg.select(["date", "heading", "newstype"]))

Feature view created successfully, explore it at 
https://hopsworks0.logicalclocks.com/p/122/fs/70/fv/news/version/1


In [23]:
# fv.find_neighbors(model.encode(news_description), k=5)

In [24]:
fv.find_neighbors(model.encode(news_description), k=5, feature=news_fg.embedding_headding)

[['1/6/2015', 'asian stocks sink euro near nine year ', 'business'],
 ['1/14/2015', 'oil extends losses as world bank cuts growth for', 'business'],
 ['1/2/2015', 'asia stocks up in new year trad', 'business'],
 ['1/22/2015', 'asia stocks mostly higher ahead of ecb meeting', 'business'],
 ['1/12/2015', 'oil falls further in asian trad', 'business']]

In [25]:
fv.find_neighbors(model.encode(news_description), k=5, feature=news_fg.embedding_body)

[['1/6/2015', 'asian stocks sink euro near nine year ', 'business'],
 ['1/19/2015', 'brent crude steady around 50', 'business'],
 ['1/16/2015', 'oil market rebounds on iea repor', 'business'],
 ['1/6/2015', 'us oil prices slip below 50 a barr', 'business'],
 ['1/22/2015', 'asia stocks mostly higher ahead of ecb meeting', 'business']]

In [26]:
# fv.find_neighbors(model.encode(news_description), k=5, 
#                   filter=news_fg.newstype == "sports",  feature=news_fg.embedding_headding)

In [27]:
# fv.find_neighbors(model.encode(news_description), k=5, 
#                   filter=((news_fg.newstype == "sports") & (news_fg.heading.like("france"))),
#                  feature=news_fg.embedding_headding)

In [28]:
fv1 = fs.get_or_create_feature_view(
    "news_cnt", version=version, 
    query=news_fg.select(["date", "heading", "newstype"]).join(view_fg.select(["view_cnt"])))

Feature view created successfully, explore it at 
https://hopsworks0.logicalclocks.com/p/122/fs/70/fv/news_cnt/version/1


In [29]:
fv1.find_neighbors(model.encode(news_description), k=5, feature=news_fg.embedding_headding)

[['1/6/2015', 'asian stocks sink euro near nine year ', 'business', 94],
 ['1/14/2015',
  'oil extends losses as world bank cuts growth for',
  'business',
  13],
 ['1/2/2015', 'asia stocks up in new year trad', 'business', 14],
 ['1/22/2015',
  'asia stocks mostly higher ahead of ecb meeting',
  'business',
  3],
 ['1/12/2015', 'oil falls further in asian trad', 'business', 17]]

In [30]:
fv1.get_feature_vector({"id1": 10})

['1/14/2015', 'tokyo stocks open 0.74 percent lower', 'business', 86]

In [33]:
news_fg.embedding_index.index_name

'122__embedding_default_project_embedding_0'

## Use project index

In [34]:
from hsfs import embedding

emb = embedding.EmbeddingIndex()
emb.add_embedding("embedding_body", len(df["embedding_body"][0]))

news_fg_1 = fs.get_or_create_feature_group(
    name="news_fg_proj",
    embedding_index=emb,
    primary_key=["id1"],
    version=version,
    online_enabled=True
)

news_fg_1.insert(df)

Feature Group created successfully, explore it at 
https://hopsworks0.logicalclocks.com/p/122/fs/70/fg/43


Uploading Dataframe: 0.00% |          | Rows 0/20 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: news_fg_proj_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://hopsworks0.logicalclocks.com/p/122/jobs/named/news_fg_proj_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f43578fddb0>, None)

In [35]:
news_fg_1.embedding_index.index_name

'122__embedding_default_project_embedding_0'

In [37]:
news_fg_1.show(n=3, online=True)

[['KARACHI: Strong bulls on Friday pulled the benchmark KSE-100 Index at Karachi Stock Exchange (KSE) and taking it across the psychological barrier of 33,000 points with a single days gain of 207 points.The investors remained active right from the opening bells of todays trade, contributing 207 points to the major index and a close at 33,324 points  a new record in Pakistans stock market history.According to stock market experts, the investors are interested in taking fresh positions in the expectations of discount rate going further down as a result of a visible decline in inflation rate.They said the stock market of Pakistan has earned the honour of being considered as one of the most successful markets of this region in 2014 during which investors booked over 25 percent profits.  \r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n',
  '1/9/2015',
  'bullish kse jumps over 33000 psychological barrier',
  'business',
  'AoAGAgAAACB1kJU/AgAAAODNUIC/AgAAAEAhRbQ/AgAAAICFcag/AgAAACA6WqK/AgAAACA